Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = '/home/garfield/tmp/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 28, 28) (20000,)
Validation set (1000, 28, 28) (1000,)
Test set (1000, 28, 28) (1000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (20000, 784) (20000, 10)
Validation set (1000, 784) (1000, 10)
Test set (1000, 784) (1000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

###  Multinomial logistic regression with L2 loss function

In [5]:
batch_size = 1000
beta = 0.1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  
  # Original loss
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regularizer = tf.nn.l2_loss(weights)
  
  # Loss with L2 regularization
  loss = tf.reduce_mean(loss + beta * regularizer)

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 328.339813
Minibatch accuracy: 8.6%
Validation accuracy: 10.5%
Minibatch loss at step 500: 1.040032
Minibatch accuracy: 80.1%
Validation accuracy: 82.3%
Minibatch loss at step 1000: 1.015078
Minibatch accuracy: 79.3%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 0.981836
Minibatch accuracy: 81.1%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 1.029584
Minibatch accuracy: 79.8%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 0.998718
Minibatch accuracy: 80.6%
Validation accuracy: 82.0%
Minibatch loss at step 3000: 0.991863
Minibatch accuracy: 80.2%
Validation accuracy: 81.6%
Test accuracy: 86.4%


### Neural network with L2 regularization

In [7]:
# LeRU with mini-batch: 

# graph definition
num_nodes = 1024
batch_size = 128
beta = 0.01
n_features = image_size * image_size
graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(dtype=np.float32, shape=[batch_size, n_features])
  tf_train_labels = tf.placeholder(dtype=np.float32, shape=[batch_size, num_labels])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
  w2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2=  tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1) 
  logits = tf.matmul(h1, w2)+b2
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # loss with l2 regularization
  reg = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
  loss = tf.reduce_mean(loss + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [8]:
# training
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3515.843994
Minibatch accuracy: 6.2%
Validation accuracy: 27.2%
Minibatch loss at step 500: 21.382870
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 1000: 0.899228
Minibatch accuracy: 82.8%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 0.649707
Minibatch accuracy: 86.7%
Validation accuracy: 84.2%
Minibatch loss at step 2000: 0.779200
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 0.825234
Minibatch accuracy: 82.8%
Validation accuracy: 84.5%
Minibatch loss at step 3000: 0.890049
Minibatch accuracy: 82.0%
Validation accuracy: 86.0%
Test accuracy: 89.4%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

** Make dataset small so we only have a few batches **

In [9]:
# training
train_dataset_small = train_dataset[:500, :]
train_labels_small = train_labels[:500]
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3423.588867
Minibatch accuracy: 12.5%
Validation accuracy: 40.4%
Minibatch loss at step 500: 21.089260
Minibatch accuracy: 100.0%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 0.492614
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 1500: 0.312221
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.298647
Minibatch accuracy: 100.0%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.288829
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 3000: 0.282555
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Test accuracy: 84.6%


Overfitting: high training accuracy but low cross validation accuracy

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:

# graph definition
num_nodes = 1024
batch_size = 128
beta = 0.01
n_features = image_size * image_size
graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(dtype=np.float32, shape=[batch_size, n_features])
  tf_train_labels = tf.placeholder(dtype=np.float32, shape=[batch_size, num_labels])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  b1 = tf.Variable(tf.zeros([num_nodes]))
  w2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  b2=  tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1) 
#   logits = tf.matmul(h1, w2)+b2
    
  # apply dropouts and calculate logits
  keep_prob = tf.placeholder("float")
  h1_dropout = tf.nn.dropout(h1, keep_prob)
  logits = tf.matmul(h1_dropout, w2) + b2
    
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # loss with l2 regularization
  reg = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
  loss = tf.reduce_mean(loss + beta * reg)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1), w2) + b2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1), w2) + b2)

In [12]:
# training
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3616.918945
Minibatch accuracy: 10.2%
Validation accuracy: 32.2%
Minibatch loss at step 500: 21.456099
Minibatch accuracy: 81.2%
Validation accuracy: 86.1%
Minibatch loss at step 1000: 0.974338
Minibatch accuracy: 82.8%
Validation accuracy: 84.9%
Minibatch loss at step 1500: 0.736956
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.873296
Minibatch accuracy: 82.0%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 0.906831
Minibatch accuracy: 82.8%
Validation accuracy: 83.6%
Minibatch loss at step 3000: 0.957785
Minibatch accuracy: 78.9%
Validation accuracy: 85.1%
Test accuracy: 89.5%


**Extreme overfitting**

In [13]:
# training
train_dataset_small = train_dataset[:500, :]
train_labels_small = train_labels[:500]
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3636.041748
Minibatch accuracy: 10.9%
Validation accuracy: 31.4%
Minibatch loss at step 500: 21.143982
Minibatch accuracy: 100.0%
Validation accuracy: 79.8%
Minibatch loss at step 1000: 0.526597
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 1500: 0.343390
Minibatch accuracy: 100.0%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.332250
Minibatch accuracy: 100.0%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 0.316504
Minibatch accuracy: 99.2%
Validation accuracy: 79.5%
Minibatch loss at step 3000: 0.314508
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Test accuracy: 84.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [20]:
import math as math

# graph definition
num_nodes1 = 1024
num_nodes2 = 512
batch_size = 128
beta = 0.001
n_features = image_size * image_size
graph = tf.Graph()
with graph.as_default():
  tf_train_dataset = tf.placeholder(dtype=np.float32, shape=[batch_size, n_features])
  tf_train_labels = tf.placeholder(dtype=np.float32, shape=[batch_size, num_labels])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder("float")  

  # Variables.
  # Without setting `stddev=math.sqrt(2.0/n_features)` the output will very poor
  w1 = tf.Variable(tf.truncated_normal([n_features, num_nodes1], stddev=math.sqrt(2.0/n_features)))
  b1 = tf.Variable(tf.zeros([num_nodes1]))
  w2 = tf.Variable(tf.truncated_normal([num_nodes1, num_nodes2], stddev=math.sqrt(2.0/num_nodes1)))
  b2=  tf.Variable(tf.zeros([num_nodes2]))
  w3 = tf.Variable(tf.truncated_normal([num_nodes2, num_labels], stddev=math.sqrt(2.0/num_nodes2)))
  b3=  tf.Variable(tf.zeros([num_labels]))
  
  # Hidden layer 1 
  logits_1 = tf.matmul(tf_train_dataset, w1) + b1
  h1 = tf.nn.relu(logits_1) 
  h1_dropout = tf.nn.dropout(h1, keep_prob)

  # Hidden layer 2
  logits_2 = tf.matmul(h1_dropout, w2) + b2
  h2 = tf.nn.relu(logits_2)
  h2_dropout = tf.nn.dropout(h2, keep_prob)

  # Output layer
  logits = tf.matmul(h2_dropout, w3) + b3   
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

  # loss with l2 regularization
  reg = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) +tf.nn.l2_loss(w3)
  loss = tf.reduce_mean(loss + beta * reg)
  
  # Optimizer.
  # Decaying learning rate
  global_step = tf.Variable(0)  # count the number of steps taken.
  start_learning_rate = 0.5
  learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 3000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training
  train_prediction = tf.nn.softmax(logits)

  # Predictions for the validation
  valid_logits_1 = tf.matmul(tf_valid_dataset, w1) + b1
  valid_relu_1 = tf.nn.relu(valid_logits_1)
  valid_logits_2 = tf.matmul(valid_relu_1, w2) + b2
  valid_relu_2 = tf.nn.relu(valid_logits_2)
  valid_logits_3 = tf.matmul(valid_relu_2, w3) + b3
  valid_prediction = tf.nn.softmax(valid_logits_3)
    
    
  # Predictions for the testing
  test_logits_1 = tf.matmul(tf_test_dataset, w1) + b1
  test_relu_1 = tf.nn.relu(test_logits_1)
  test_logits_2 = tf.matmul(test_relu_1, w2) + b2
  test_relu_2 = tf.nn.relu(test_logits_2)
  test_logits_3 = tf.matmul(test_relu_2, w3) + b3
  test_prediction = tf.nn.softmax(test_logits_3)

In [22]:
# training
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.689597
Minibatch accuracy: 10.9%
Validation accuracy: 34.6%
Minibatch loss at step 500: 1.297666
Minibatch accuracy: 83.6%
Validation accuracy: 86.5%
Minibatch loss at step 1000: 1.039829
Minibatch accuracy: 84.4%
Validation accuracy: 87.2%
Minibatch loss at step 1500: 0.803166
Minibatch accuracy: 87.5%
Validation accuracy: 87.1%
Minibatch loss at step 2000: 0.748158
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 2500: 0.724465
Minibatch accuracy: 90.6%
Validation accuracy: 88.0%
Minibatch loss at step 3000: 0.740478
Minibatch accuracy: 85.9%
Validation accuracy: 87.2%
Minibatch loss at step 3500: 0.512150
Minibatch accuracy: 91.4%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.409654
Minibatch accuracy: 94.5%
Validation accuracy: 88.1%
Minibatch loss at step 4500: 0.668459
Minibatch accuracy: 86.7%
Validation accuracy: 86.3%
Minibatch loss at step 5000: 0.658810
Minibatch accuracy: 86.7%
Validation accurac